# Проект для «Викишоп» с BERT
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75.

# Подключение и настройка необходимых библиотек

In [1]:
#pip install transformers
#pip install tokenization
#pip install torch

In [2]:
# отключение предупреждений
import warnings; warnings.filterwarnings('ignore', category=Warning)

In [24]:
import numpy as np
import pandas as pd
import re

import torch
import transformers as ppb 
import tokenization

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.dummy import DummyClassifier

In [6]:
try:
    data = pd.read_csv('toxic_comments.csv')
except FileNotFoundError:
    data = pd.read_csv('https://code.s3/datasets/toxic_comments.csv')

In [7]:
# посмотрим, корректно ли загрузились наши данные
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [8]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


# Предобработка текста

In [9]:
# взглянув на датасет заметили, что перенос строки отображается как "
, заменим его на пробел
data['text'] = data['text'].str.replace('\n', ' ')

In [10]:
# также почистим его от символов
def clear_text(row):
    row = re.sub(r'[^a-zA-Z ]', ' ', row) 
    row = row.split()
    row = " ".join(row)
    return row

# Bert за работой

In [29]:
# чтобы избежать смерти нашей оперативной памяти, преобразовывать будем 400 экземпляров
batch = data.iloc[:400]

In [30]:
# очистим их от словесного мусора
batch['text'] = batch['text'].apply(clear_text)

Использовать будем DistilBERT -- это версия BERT, которая меньше, быстрее и легче оригинала

In [31]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# можно использовать и саму BERT
# model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

## Токенизация
Наш первый шаг -- токенизация, то есть нам нужно разбить предложений на слова и подслова в формате, удобном для BERT.

In [32]:
# BERT работает только с текстами длиной не более 512 токенов
tokenized = batch['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=100, truncation=True))

## Padding
После токенизации каждое предложение представлено в виде списка токенов. Мы хотим, чтобы BERT обрабатывал все наши примеры одновременно, по этой причине нам нужно сделать векторы одинакового размера. Чтобы мы  представить входные данные как один двумерный массив, а не как список списков разной длины, заполним недостающие значения нулями.

In [33]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

## Masking
Чтобы не запутать BERT, создадим еще одну переменную, которая будет игнорировать (маскировать) добавленное заполнение.

In [34]:
attention_mask = np.where(padded != 0, 1, 0)

## Обучение модели
Функция model() запускает наши предложения через BERT. Результаты обработки будут возвращены в last_hidden_states.

In [35]:
input_ids = torch.tensor(padded).to(torch.int64)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

При выполнении классификации BERT добавляет токен [CLS] в начале каждого предложения.  Поэтому результат, соответствующий первому токену каждого предложения, мы и подадим в качестве признака для следующей модели -- логистической регрессии

In [36]:
features = last_hidden_states[0][:,0,:].numpy()

In [37]:
labels = batch.iloc[:, 1].values

In [38]:
# разделим на тестовую и тренировочную выборку
features_train, features_test, labels_train, label_test = train_test_split(features, labels, test_size=0.75)

In [39]:
# обучим модель
lr = LogisticRegression().fit(features_train, labels_train)

In [40]:
y_pred = lr.predict(features_test)
print(f1_score(label_test, y_pred))

0.6545454545454547


In [42]:
from sklearn.model_selection import GridSearchCV

parameters = {'C': np.linspace(0.0001, 100, 20)}
f1 = make_scorer(f1_score)

grid_search = GridSearchCV(LogisticRegression(), parameters, scoring=f1)
grid_search.fit(features_train, labels_train)

best_grid = grid_search.best_estimator_
grid_pred = best_grid.predict(features_test)
print(f1_score(label_test, grid_pred))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.6031746031746031


**Вывод:**
Итак, нам удалось выполнить небольшую предобработку текста небольшой выборки из 400 предложений (иначе комп решает умереть). Использовав DistilBERT и линейную регрессию мы смогли получить F1 0.65 на тестовой выборке. При увеличении размера выборки модель будет лучше обучать и результат F1 будет также меняться в лучшую для нас сторону. 